# 62. ボリュームレンダリング - 連続3D表現の可視化

**難易度**: ★★★★☆（中上級）

## 学習目標

このノートブックを完了すると、以下ができるようになります：

- [ ] ボリュームレンダリングの物理的原理を理解する
- [ ] 透過率と不透明度の計算方法を実装できる
- [ ] レイに沿った色の積分（離散近似）を実装できる
- [ ] アルファ合成とボリュームレンダリングの関係を理解する
- [ ] NeRFで使用されるレンダリング方程式を導出できる

## 前提知識

- Ray Castingと座標系（Notebook 61）
- 積分の基礎
- 指数関数の性質

## 目次

1. [ボリュームレンダリングとは](#1-ボリュームレンダリングとは)
2. [光の伝播モデル](#2-光の伝播モデル)
3. [ボリュームレンダリング方程式](#3-ボリュームレンダリング方程式)
4. [離散化と数値計算](#4-離散化と数値計算)
5. [実装とデモ](#5-実装とデモ)
6. [NeRFとの関連](#6-nerfとの関連)
7. [まとめとセルフチェック](#7-まとめとセルフチェック)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import Tuple, Callable
import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(precision=4, suppress=True)
print("ライブラリの読み込み完了")

---

## 1. ボリュームレンダリングとは

### 1.1 サーフェスレンダリング vs ボリュームレンダリング

| 特徴 | サーフェスレンダリング | ボリュームレンダリング |
|------|----------------------|----------------------|
| 表現 | 明確な表面（メッシュ） | 連続的な密度場 |
| 交差判定 | レイと面の交差 | レイに沿った積分 |
| 半透明 | 追加処理が必要 | 自然にサポート |
| 応用例 | ゲーム、CAD | 医療画像、雲、NeRF |

### 1.2 ボリュームレンダリングの応用

- **医療画像**: CTスキャン、MRIの可視化
- **科学可視化**: 流体シミュレーション、気象データ
- **コンピュータグラフィックス**: 雲、煙、霧
- **Neural Rendering**: NeRF、3D Gaussian Splatting

In [ ]:
# サーフェス vs ボリュームの概念図
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# サーフェスレンダリング
ax = axes[0]
ax.set_title('サーフェスレンダリング\n（明確な境界）', fontsize=12)

# 球の輪郭
theta = np.linspace(0, 2*np.pi, 100)
x_circle = 0.5 + 0.3 * np.cos(theta)
y_circle = 0.5 + 0.3 * np.sin(theta)
ax.fill(x_circle, y_circle, color='blue', alpha=0.8)
ax.plot(x_circle, y_circle, 'k-', linewidth=2)

# レイ
ax.annotate('', xy=(0.5, 0.5), xytext=(0.1, 0.8),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))
ax.scatter([0.5 - 0.3*np.cos(np.pi/4)], [0.5 + 0.3*np.sin(np.pi/4)], 
           s=100, c='green', zorder=10, label='交差点')
ax.text(0.15, 0.75, 'レイ', color='red', fontsize=11)

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')
ax.legend()
ax.axis('off')

# ボリュームレンダリング
ax = axes[1]
ax.set_title('ボリュームレンダリング\n（連続的な密度）', fontsize=12)

# 密度場（グラデーション）
x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100)
X, Y = np.meshgrid(x, y)
density = np.exp(-((X-0.5)**2 + (Y-0.5)**2) / (2*0.15**2))
ax.imshow(density, extent=[0, 1, 0, 1], origin='lower', 
          cmap='Blues', alpha=0.8)

# レイとサンプル点
ray_x = np.linspace(0.1, 0.9, 10)
ray_y = 0.8 - 0.4 * (ray_x - 0.1)
ax.plot(ray_x, ray_y, 'r-', linewidth=2, label='レイ')
ax.scatter(ray_x, ray_y, c='green', s=50, zorder=10, label='サンプル点')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')
ax.legend(loc='upper right')
ax.axis('off')

plt.tight_layout()
plt.show()

print("サーフェスレンダリング: レイと表面の交差点で色を決定")
print("ボリュームレンダリング: レイに沿った全ての点の寄与を積分")

---

## 2. 光の伝播モデル

### 2.1 吸収（Absorption）

媒質を通過する光は吸収により減衰します。微小距離 $dt$ を進むとき：

$$
dI = -\sigma(t) I(t) dt
$$

ここで：
- $I(t)$: 位置 $t$ での光の強度
- $\sigma(t)$: 吸収係数（密度）

### 2.2 透過率（Transmittance）

区間 $[t_n, t]$ での透過率 $T(t)$ は：

$$
T(t) = \exp\left( -\int_{t_n}^{t} \sigma(s) ds \right)
$$

- $T(t) = 1$: 完全に透明（光が100%通過）
- $T(t) = 0$: 完全に不透明（光が全く通過しない）

### 2.3 発光（Emission）

各点は色 $\mathbf{c}(t)$ を放射します。最終的な色は吸収と発光の組み合わせです。

In [ ]:
# 透過率の可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

t = np.linspace(0, 5, 200)

# 密度パターン
density_patterns = [
    ('一定密度 σ=0.5', lambda t: 0.5 * np.ones_like(t)),
    ('線形増加 σ=0.2t', lambda t: 0.2 * t),
    ('ガウシアン密度', lambda t: np.exp(-((t-2.5)**2)/0.5)),
]

for ax, (name, sigma_func) in zip(axes, density_patterns):
    sigma = sigma_func(t)
    
    # 累積密度（積分）
    dt = t[1] - t[0]
    cumulative_sigma = np.cumsum(sigma) * dt
    
    # 透過率
    transmittance = np.exp(-cumulative_sigma)
    
    ax.fill_between(t, 0, sigma, alpha=0.3, color='blue', label='密度 σ(t)')
    ax.plot(t, sigma, 'b-', linewidth=2)
    ax.plot(t, transmittance, 'r-', linewidth=2, label='透過率 T(t)')
    
    ax.set_xlabel('t (レイに沿った距離)')
    ax.set_ylabel('値')
    ax.set_title(name)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 1.5)

plt.tight_layout()
plt.show()

print("観察:")
print("- 密度が高いほど透過率は急速に減少")
print("- ガウシアン密度: 物体の中心付近で急激に減少")

---

## 3. ボリュームレンダリング方程式

### 3.1 連続形式

最終的なピクセル色 $C$ は以下の積分で与えられます：

$$
C(\mathbf{r}) = \int_{t_n}^{t_f} T(t) \sigma(\mathbf{r}(t)) \mathbf{c}(\mathbf{r}(t), \mathbf{d}) dt
$$

ここで：
- $T(t) = \exp\left(-\int_{t_n}^{t} \sigma(\mathbf{r}(s)) ds\right)$: 透過率
- $\sigma(\mathbf{r}(t))$: 位置 $\mathbf{r}(t)$ での密度
- $\mathbf{c}(\mathbf{r}(t), \mathbf{d})$: 位置と視線方向に依存する色

### 3.2 物理的解釈

$$
C = \int T(t) \cdot \sigma(t) \cdot \mathbf{c}(t) dt
$$

| 項 | 意味 |
|---|------|
| $T(t)$ | 「この点までどれだけ光が届くか」（遮蔽されていない確率）|
| $\sigma(t)$ | 「この点にどれだけ物質があるか」（密度）|
| $\mathbf{c}(t)$ | 「この点の色は何か」|

In [ ]:
# ボリュームレンダリング方程式の可視化
fig = plt.figure(figsize=(14, 10))

t = np.linspace(0, 5, 200)
dt = t[1] - t[0]

# 2つの物体（異なる密度と色）
sigma1 = 1.5 * np.exp(-((t-1.5)**2)/0.3)  # 前方の物体
sigma2 = 2.0 * np.exp(-((t-3.5)**2)/0.2)  # 後方の物体
sigma = sigma1 + sigma2

# 色（RGB）
color1 = np.array([1.0, 0.3, 0.3])  # 赤っぽい
color2 = np.array([0.3, 0.3, 1.0])  # 青っぽい

# 各点での色
colors = np.zeros((len(t), 3))
for i in range(len(t)):
    w1 = sigma1[i] / (sigma[i] + 1e-8)
    w2 = sigma2[i] / (sigma[i] + 1e-8)
    colors[i] = w1 * color1 + w2 * color2

# 透過率
cumulative_sigma = np.cumsum(sigma) * dt
transmittance = np.exp(-cumulative_sigma)

# 寄与 = T * sigma * c
contribution = transmittance[:, None] * sigma[:, None] * colors

# Plot 1: 密度分布
ax1 = fig.add_subplot(321)
ax1.fill_between(t, 0, sigma1, alpha=0.5, color='red', label='物体1')
ax1.fill_between(t, sigma1, sigma1+sigma2, alpha=0.5, color='blue', label='物体2')
ax1.plot(t, sigma, 'k-', linewidth=2, label='合計密度')
ax1.set_xlabel('t')
ax1.set_ylabel('密度 σ(t)')
ax1.set_title('Step 1: 密度分布')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: 透過率
ax2 = fig.add_subplot(322)
ax2.plot(t, transmittance, 'g-', linewidth=2)
ax2.fill_between(t, 0, transmittance, alpha=0.3, color='green')
ax2.set_xlabel('t')
ax2.set_ylabel('T(t)')
ax2.set_title('Step 2: 透過率 T(t) = exp(-∫σds)')
ax2.grid(True, alpha=0.3)

# Plot 3: 各点の寄与
ax3 = fig.add_subplot(323)
ax3.plot(t, contribution[:, 0], 'r-', linewidth=2, label='R寄与')
ax3.plot(t, contribution[:, 1], 'g-', linewidth=2, label='G寄与')
ax3.plot(t, contribution[:, 2], 'b-', linewidth=2, label='B寄与')
ax3.set_xlabel('t')
ax3.set_ylabel('寄与')
ax3.set_title('Step 3: 各点の寄与 T(t)·σ(t)·c(t)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: 累積色
ax4 = fig.add_subplot(324)
cumulative_color = np.cumsum(contribution, axis=0) * dt
ax4.plot(t, cumulative_color[:, 0], 'r-', linewidth=2, label='累積R')
ax4.plot(t, cumulative_color[:, 1], 'g-', linewidth=2, label='累積G')
ax4.plot(t, cumulative_color[:, 2], 'b-', linewidth=2, label='累積B')
ax4.set_xlabel('t')
ax4.set_ylabel('累積色')
ax4.set_title('Step 4: 累積色（積分）')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Plot 5: 最終色
ax5 = fig.add_subplot(325)
final_color = np.clip(cumulative_color[-1], 0, 1)
ax5.imshow([[final_color]], aspect='auto')
ax5.set_title(f'最終色: RGB = ({final_color[0]:.2f}, {final_color[1]:.2f}, {final_color[2]:.2f})')
ax5.axis('off')

# Plot 6: レイの図解
ax6 = fig.add_subplot(326)
ax6.set_title('レイに沿った色の積分', fontsize=12)

# レイを描画
ax6.arrow(0.05, 0.5, 0.85, 0, head_width=0.05, head_length=0.03, 
          fc='black', ec='black')
ax6.text(0.02, 0.5, 'カメラ', ha='right', va='center')

# 物体を描画
for center, color, alpha in [(0.3, 'red', 0.6), (0.7, 'blue', 0.8)]:
    circle = plt.Circle((center, 0.5), 0.1, color=color, alpha=alpha)
    ax6.add_patch(circle)

ax6.set_xlim(0, 1)
ax6.set_ylim(0, 1)
ax6.set_aspect('equal')
ax6.axis('off')

plt.tight_layout()
plt.show()

---

## 4. 離散化と数値計算

### 4.1 離散近似

連続積分を離散的なサンプル点で近似します：

$$
\hat{C} = \sum_{i=1}^{N} T_i \alpha_i \mathbf{c}_i
$$

ここで：
- $\alpha_i = 1 - \exp(-\sigma_i \delta_i)$: 不透明度（opacity）
- $\delta_i = t_{i+1} - t_i$: サンプル間の距離
- $T_i = \prod_{j=1}^{i-1}(1 - \alpha_j)$: 累積透過率

### 4.2 アルファ合成との関係

この離散化は、標準的な **アルファブレンディング（front-to-back compositing）** と等価です：

```
C = 0, T = 1
for i in 1..N:
    C = C + T * α_i * c_i
    T = T * (1 - α_i)
```

In [ ]:
def volume_render_discrete(sigmas: np.ndarray, colors: np.ndarray, 
                          deltas: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    離散ボリュームレンダリング
    
    Parameters:
    -----------
    sigmas : np.ndarray, shape (N,)
        各サンプル点での密度
    colors : np.ndarray, shape (N, 3)
        各サンプル点での色 (RGB)
    deltas : np.ndarray, shape (N,)
        サンプル間の距離
        
    Returns:
    --------
    final_color : np.ndarray, shape (3,)
        レンダリングされた最終色
    weights : np.ndarray, shape (N,)
        各サンプルの重み (T_i * alpha_i)
    transmittance : np.ndarray, shape (N,)
        各点での透過率
    """
    N = len(sigmas)
    
    # 不透明度: α = 1 - exp(-σδ)
    alphas = 1.0 - np.exp(-sigmas * deltas)
    
    # 累積透過率: T_i = Π(1 - α_j) for j < i
    # T_1 = 1, T_2 = (1-α_1), T_3 = (1-α_1)(1-α_2), ...
    transmittance = np.ones(N)
    for i in range(1, N):
        transmittance[i] = transmittance[i-1] * (1 - alphas[i-1])
    
    # 重み: w_i = T_i * α_i
    weights = transmittance * alphas
    
    # 最終色: C = Σ w_i * c_i
    final_color = np.sum(weights[:, None] * colors, axis=0)
    
    return final_color, weights, transmittance

# テスト: 2つの半透明物体
N = 64
t_vals = np.linspace(0, 5, N)
deltas = np.ones(N) * (t_vals[1] - t_vals[0])

# 密度（2つのガウシアン）
sigmas = 2.0 * np.exp(-((t_vals-1.5)**2)/0.3) + \
         3.0 * np.exp(-((t_vals-3.5)**2)/0.2)

# 色（前方: 赤、後方: 青）
colors = np.zeros((N, 3))
for i, t in enumerate(t_vals):
    if t < 2.5:
        colors[i] = [1.0, 0.2, 0.2]  # 赤
    else:
        colors[i] = [0.2, 0.2, 1.0]  # 青

final_color, weights, transmittance = volume_render_discrete(sigmas, colors, deltas)

print(f"最終色 (RGB): [{final_color[0]:.3f}, {final_color[1]:.3f}, {final_color[2]:.3f}]")
print(f"重みの合計: {np.sum(weights):.3f}")

In [ ]:
# 離散ボリュームレンダリングの可視化
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 密度と色
ax = axes[0, 0]
ax.fill_between(t_vals, 0, sigmas, alpha=0.3, color='gray')
ax.plot(t_vals, sigmas, 'k-', linewidth=2, label='密度 σ')
# 色をオーバーレイ
for i in range(N-1):
    ax.axvspan(t_vals[i], t_vals[i+1], alpha=0.5, 
               color=colors[i], ymin=0, ymax=sigmas[i]/sigmas.max())
ax.set_xlabel('t')
ax.set_ylabel('密度 σ(t)')
ax.set_title('密度分布と色')
ax.legend()
ax.grid(True, alpha=0.3)

# 不透明度と透過率
ax = axes[0, 1]
alphas = 1.0 - np.exp(-sigmas * deltas)
ax.plot(t_vals, alphas, 'b-', linewidth=2, label='不透明度 α')
ax.plot(t_vals, transmittance, 'g-', linewidth=2, label='透過率 T')
ax.set_xlabel('t')
ax.set_ylabel('値')
ax.set_title('不透明度 α = 1-exp(-σδ) と 透過率 T')
ax.legend()
ax.grid(True, alpha=0.3)

# 重み
ax = axes[1, 0]
ax.fill_between(t_vals, 0, weights, alpha=0.5, color='purple')
ax.plot(t_vals, weights, 'purple', linewidth=2, label='重み w = T·α')
ax.set_xlabel('t')
ax.set_ylabel('重み')
ax.set_title(f'各点の重み（合計 = {np.sum(weights):.3f}）')
ax.legend()
ax.grid(True, alpha=0.3)

# 重み付き色の累積
ax = axes[1, 1]
weighted_colors = weights[:, None] * colors
cumulative_rgb = np.cumsum(weighted_colors, axis=0)
ax.plot(t_vals, cumulative_rgb[:, 0], 'r-', linewidth=2, label='累積 R')
ax.plot(t_vals, cumulative_rgb[:, 1], 'g-', linewidth=2, label='累積 G')
ax.plot(t_vals, cumulative_rgb[:, 2], 'b-', linewidth=2, label='累積 B')
ax.axhline(final_color[0], color='r', linestyle='--', alpha=0.5)
ax.axhline(final_color[2], color='b', linestyle='--', alpha=0.5)
ax.set_xlabel('t')
ax.set_ylabel('累積色')
ax.set_title(f'累積色 → 最終色')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 最終色を表示
fig, ax = plt.subplots(figsize=(4, 2))
ax.imshow([[np.clip(final_color, 0, 1)]])
ax.set_title(f'最終レンダリング色\nRGB = ({final_color[0]:.2f}, {final_color[1]:.2f}, {final_color[2]:.2f})')
ax.axis('off')
plt.show()

### 4.3 重みの解釈

重み $w_i = T_i \alpha_i$ は、「サンプル点 $i$ がどれだけ最終色に寄与するか」を表します：

- **$T_i$ が大きい**: その点までの遮蔽が少ない（前方の物体が薄い）
- **$\alpha_i$ が大きい**: その点の密度が高い（物質が多い）
- 両方が大きいときに最大の寄与

この重みは、NeRFで「期待される深度」の計算などにも使用されます。

In [ ]:
# 期待深度の計算
expected_depth = np.sum(weights * t_vals)

print(f"期待される深度: {expected_depth:.3f}")

# 可視化
fig, ax = plt.subplots(figsize=(10, 4))
ax.fill_between(t_vals, 0, weights, alpha=0.5, color='purple', label='重み分布')
ax.axvline(expected_depth, color='red', linewidth=2, linestyle='--', 
           label=f'期待深度 = {expected_depth:.2f}')
ax.axvline(1.5, color='orange', linewidth=1, linestyle=':', label='物体1の中心 (t=1.5)')
ax.axvline(3.5, color='blue', linewidth=1, linestyle=':', label='物体2の中心 (t=3.5)')
ax.set_xlabel('t (深度)')
ax.set_ylabel('重み')
ax.set_title('重み分布と期待深度')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

print("\n観察:")
print("- 期待深度は重みの加重平均")
print("- 前方の物体（赤）の方が重みが大きいため、期待深度は前寄り")

---

## 5. 実装とデモ

### 5.1 シンプルな3Dシーンのボリュームレンダリング

In [ ]:
class VolumeScene:
    """ボリュームレンダリング用のシンプルなシーン"""
    
    def __init__(self):
        # 球のリスト: (center, radius, density, color)
        self.spheres = [
            {'center': np.array([0, 0, 3]), 'radius': 0.8, 
             'density': 5.0, 'color': np.array([1.0, 0.3, 0.3])},
            {'center': np.array([0.5, 0.3, 5]), 'radius': 1.0, 
             'density': 3.0, 'color': np.array([0.3, 1.0, 0.3])},
            {'center': np.array([-0.3, -0.2, 4]), 'radius': 0.6, 
             'density': 4.0, 'color': np.array([0.3, 0.3, 1.0])},
        ]
    
    def query(self, points: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        各点での密度と色を取得
        
        Parameters:
        -----------
        points : np.ndarray, shape (N, 3)
            クエリ点
            
        Returns:
        --------
        sigmas : np.ndarray, shape (N,)
        colors : np.ndarray, shape (N, 3)
        """
        N = len(points)
        sigmas = np.zeros(N)
        colors = np.zeros((N, 3))
        
        for sphere in self.spheres:
            # 各点から球の中心への距離
            dists = np.linalg.norm(points - sphere['center'], axis=1)
            
            # ガウシアン密度場
            sphere_sigma = sphere['density'] * np.exp(
                -(dists / sphere['radius'])**2
            )
            
            # 密度と色を加算
            sigmas += sphere_sigma
            colors += sphere_sigma[:, None] * sphere['color']
        
        # 色を正規化（密度で重み付け平均）
        mask = sigmas > 1e-8
        colors[mask] /= sigmas[mask, None]
        colors[~mask] = 0
        
        return sigmas, colors

# シーンの作成
scene = VolumeScene()
print(f"シーン内の球の数: {len(scene.spheres)}")

In [ ]:
def render_image(scene: VolumeScene, H: int, W: int, 
                 focal: float, near: float, far: float,
                 n_samples: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    ボリュームレンダリングで画像を生成
    
    Returns:
    --------
    image : np.ndarray, shape (H, W, 3)
    depth_map : np.ndarray, shape (H, W)
    """
    image = np.zeros((H, W, 3))
    depth_map = np.zeros((H, W))
    
    # カメラ中心（原点）
    camera_origin = np.array([0, 0, 0])
    
    # サンプリング位置
    t_vals = np.linspace(near, far, n_samples)
    deltas = np.ones(n_samples) * (far - near) / n_samples
    
    for i in range(H):
        for j in range(W):
            # ピクセル座標からレイ方向を計算
            # (u, v) -> (x, y, z) カメラ座標
            x = (j - W/2) / focal
            y = (i - H/2) / focal
            direction = np.array([x, y, 1.0])
            direction = direction / np.linalg.norm(direction)
            
            # レイに沿ったサンプル点
            points = camera_origin + t_vals[:, None] * direction
            
            # シーンへのクエリ
            sigmas, colors = scene.query(points)
            
            # ボリュームレンダリング
            final_color, weights, _ = volume_render_discrete(sigmas, colors, deltas)
            
            image[i, j] = np.clip(final_color, 0, 1)
            depth_map[i, j] = np.sum(weights * t_vals)
    
    return image, depth_map

# 小さい解像度でテスト
print("レンダリング中...")
H, W = 64, 64
image, depth_map = render_image(
    scene, H, W, 
    focal=50, near=1.0, far=8.0, n_samples=64
)
print("完了!")

In [ ]:
# レンダリング結果の表示
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax = axes[0]
ax.imshow(image)
ax.set_title('ボリュームレンダリング結果')
ax.axis('off')

ax = axes[1]
im = ax.imshow(depth_map, cmap='viridis')
ax.set_title('深度マップ')
ax.axis('off')
plt.colorbar(im, ax=ax, label='深度')

plt.tight_layout()
plt.show()

print("観察:")
print("- 半透明の球が重なり合って見える")
print("- 前方の赤い球が最も明るい（透過率が高い）")
print("- 深度マップは期待深度を表示")

In [ ]:
# より高解像度でレンダリング
print("高解像度レンダリング中...（少し時間がかかります）")
H_hi, W_hi = 128, 128
image_hi, depth_hi = render_image(
    scene, H_hi, W_hi,
    focal=100, near=1.0, far=8.0, n_samples=96
)
print("完了!")

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax = axes[0]
ax.imshow(image_hi)
ax.set_title('高解像度ボリュームレンダリング (128x128)')
ax.axis('off')

ax = axes[1]
im = ax.imshow(depth_hi, cmap='plasma')
ax.set_title('深度マップ')
ax.axis('off')
plt.colorbar(im, ax=ax, label='深度')

plt.tight_layout()
plt.show()

### 5.2 密度パラメータの影響

In [ ]:
# 密度の影響を比較
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

density_values = [1.0, 3.0, 10.0, 50.0]

for ax, density in zip(axes, density_values):
    # 密度を変更したシーン
    test_scene = VolumeScene()
    for sphere in test_scene.spheres:
        sphere['density'] = density
    
    # レンダリング
    img, _ = render_image(test_scene, 64, 64, focal=50, 
                          near=1.0, far=8.0, n_samples=64)
    
    ax.imshow(img)
    ax.set_title(f'密度 σ = {density}')
    ax.axis('off')

plt.suptitle('密度パラメータの影響', fontsize=14)
plt.tight_layout()
plt.show()

print("観察:")
print("- 低密度: 透明度が高く、後ろの物体が見える")
print("- 高密度: 不透明になり、表面レンダリングに近づく")

---

## 6. NeRFとの関連

### 6.1 NeRFのレンダリング

NeRFは以下を学習します：

$$
F_\theta: (\mathbf{x}, \mathbf{d}) \rightarrow (\mathbf{c}, \sigma)
$$

- 入力: 3D位置 $\mathbf{x}$ と視線方向 $\mathbf{d}$
- 出力: 色 $\mathbf{c}$ と密度 $\sigma$

そして、このNotebookで学んだボリュームレンダリングを使用して画像を生成します。

### 6.2 微分可能レンダリング

ボリュームレンダリング方程式は**完全に微分可能**です：

1. ネットワーク出力 $(\sigma, \mathbf{c})$ に関する勾配を計算可能
2. PyTorch/JAXで自動微分が使える
3. レンダリング誤差からネットワークを学習できる

$$
\mathcal{L} = \|\hat{C} - C_{gt}\|^2
$$

In [ ]:
# NeRFパイプラインの擬似コード
print("""
# NeRFの学習パイプライン（擬似コード）

for epoch in range(num_epochs):
    # 1. ランダムなレイをサンプル
    rays_o, rays_d, target_rgb = sample_rays(images, poses)
    
    # 2. レイに沿って点をサンプル
    t_vals = stratified_sampling(near, far, n_samples)
    points = rays_o + t_vals * rays_d
    
    # 3. ネットワークでクエリ
    sigmas, colors = network(positional_encode(points), rays_d)
    
    # 4. ボリュームレンダリング
    rendered_rgb = volume_render(sigmas, colors, t_vals)
    
    # 5. 損失計算と逆伝播
    loss = mse_loss(rendered_rgb, target_rgb)
    loss.backward()
    optimizer.step()
""")

In [ ]:
# NeRFの概念図
fig = plt.figure(figsize=(16, 5))

# 1. 入力画像
ax1 = fig.add_subplot(151)
ax1.set_title('入力画像群', fontsize=10)
for i in range(3):
    rect = plt.Rectangle((0.1 + i*0.25, 0.2), 0.2, 0.3, 
                         fill=True, facecolor=f'C{i}', alpha=0.7,
                         edgecolor='black')
    ax1.add_patch(rect)
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
ax1.axis('off')

# 2. レイ生成
ax2 = fig.add_subplot(152)
ax2.set_title('レイ生成', fontsize=10)
ax2.scatter([0.2], [0.5], s=100, c='blue', marker='o')
for angle in np.linspace(-0.3, 0.3, 5):
    ax2.arrow(0.2, 0.5, 0.6, angle, head_width=0.03, 
              head_length=0.05, fc='red', ec='red', alpha=0.7)
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.axis('off')

# 3. MLPクエリ
ax3 = fig.add_subplot(153)
ax3.set_title('MLP F_θ(x,d)→(c,σ)', fontsize=10)
# 簡略化されたMLP図
layers = [5, 8, 8, 4]
x_pos = np.linspace(0.15, 0.85, len(layers))
for i, (x, n) in enumerate(zip(x_pos, layers)):
    y_pos = np.linspace(0.3, 0.7, n)
    ax3.scatter([x]*n, y_pos, s=50, c='purple')
ax3.text(0.1, 0.5, '(x,d)', ha='right', fontsize=9)
ax3.text(0.9, 0.5, '(c,σ)', ha='left', fontsize=9)
ax3.set_xlim(0, 1)
ax3.set_ylim(0, 1)
ax3.axis('off')

# 4. ボリュームレンダリング
ax4 = fig.add_subplot(154)
ax4.set_title('ボリュームレンダリング', fontsize=10)
# レイ上のサンプル点
x_ray = np.linspace(0.1, 0.9, 8)
colors_ray = plt.cm.viridis(np.linspace(0.2, 0.8, 8))
for x, c in zip(x_ray, colors_ray):
    ax4.add_patch(plt.Rectangle((x-0.03, 0.4), 0.06, 0.2, 
                                facecolor=c, alpha=0.7))
ax4.arrow(0.1, 0.5, 0.75, 0, head_width=0.05, head_length=0.05,
          fc='black', ec='black')
ax4.text(0.5, 0.25, 'Σ T_i α_i c_i', ha='center', fontsize=10)
ax4.set_xlim(0, 1)
ax4.set_ylim(0, 1)
ax4.axis('off')

# 5. 出力
ax5 = fig.add_subplot(155)
ax5.set_title('出力（新視点画像）', fontsize=10)
# グラデーション的な画像
gradient = np.zeros((20, 20, 3))
for i in range(20):
    for j in range(20):
        gradient[i, j] = [0.3 + 0.4*i/20, 0.2 + 0.3*j/20, 0.5]
ax5.imshow(gradient, extent=[0.2, 0.8, 0.2, 0.8])
ax5.set_xlim(0, 1)
ax5.set_ylim(0, 1)
ax5.axis('off')

# 矢印を追加
for i in range(4):
    fig.text(0.18 + i*0.2, 0.5, '→', fontsize=20, ha='center', va='center')

plt.tight_layout()
plt.show()

---

## 7. まとめとセルフチェック

### 学習内容の要約

| トピック | 重要なポイント |
|---------|---------------|
| ボリュームレンダリング | 連続的な密度場をレイに沿って積分 |
| 透過率 | $T(t) = \exp(-\int \sigma ds)$、累積的な遮蔽 |
| 離散化 | $\alpha_i = 1 - \exp(-\sigma_i \delta_i)$、アルファ合成 |
| 重み | $w_i = T_i \alpha_i$、各点の寄与度 |
| NeRF | ボリュームレンダリング + ニューラルネットワーク |

In [ ]:
# 重要な数式のまとめ
print("""
==================== 重要な数式 ====================

1. 連続ボリュームレンダリング方程式:
   C = ∫ T(t) σ(t) c(t) dt
   
2. 透過率:
   T(t) = exp(-∫ σ(s) ds)
   
3. 離散化（不透明度）:
   α_i = 1 - exp(-σ_i δ_i)
   
4. 離散化（累積透過率）:
   T_i = Π_{j<i} (1 - α_j)
   
5. 離散化（最終色）:
   C = Σ T_i α_i c_i
   
6. 期待深度:
   d = Σ w_i t_i, where w_i = T_i α_i

======================================================
""")

In [ ]:
# セルフチェッククイズ
print("="*60)
print("セルフチェッククイズ")
print("="*60)

questions = [
    {
        "q": "Q1: 透過率 T(t) = 1 が意味することは？",
        "options": ["a) 完全に不透明", "b) 完全に透明", "c) 半透明", "d) 密度が最大"],
        "answer": "b",
        "explanation": "T(t)=1 は光が100%通過することを意味し、完全に透明な状態です。"
    },
    {
        "q": "Q2: 不透明度 α の計算式は？",
        "options": ["a) α = σδ", "b) α = exp(-σδ)", "c) α = 1 - exp(-σδ)", "d) α = 1/σδ"],
        "answer": "c",
        "explanation": "不透明度は α = 1 - exp(-σδ) で計算されます。exp(-σδ) は透過率です。"
    },
    {
        "q": "Q3: 重み w_i = T_i α_i が大きくなる条件は？",
        "options": ["a) 前方の密度が高い", "b) その点の密度が低い", 
                   "c) 前方の遮蔽が少なく、その点の密度が高い", "d) 全ての点で均一"],
        "answer": "c",
        "explanation": "T_i（前方の遮蔽が少ない）と α_i（その点の密度が高い）の両方が大きいときに重みが大きくなります。"
    },
    {
        "q": "Q4: ボリュームレンダリングとサーフェスレンダリングの主な違いは？",
        "options": ["a) 計算速度", "b) 連続的な密度場 vs 明確な境界", 
                   "c) 2D vs 3D", "d) カラー vs モノクロ"],
        "answer": "b",
        "explanation": "ボリュームレンダリングは連続的な密度場を扱い、サーフェスレンダリングは明確な表面境界を持ちます。"
    },
    {
        "q": "Q5: NeRFでボリュームレンダリングを使う主な理由は？",
        "options": ["a) 計算が高速", "b) メモリ効率が良い", 
                   "c) 微分可能で勾配ベースの学習が可能", "d) 実装が簡単"],
        "answer": "c",
        "explanation": "ボリュームレンダリングは完全に微分可能なため、レンダリング誤差からネットワークを勾配ベースで学習できます。"
    }
]

for q in questions:
    print(f"\n{q['q']}")
    for opt in q['options']:
        print(f"  {opt}")

print("\n" + "="*60)
print("解答は下のセルを実行してください")
print("="*60)

In [ ]:
# 解答表示
print("解答と解説:")
print("="*60)
for i, q in enumerate(questions):
    print(f"\nQ{i+1}: 正解は {q['answer']}")
    print(f"   {q['explanation']}")

### 次のステップ

このNotebookでボリュームレンダリングの原理を学びました。次のNotebookでは：

- **63. NeRF入門**: Neural Radiance Fieldsの完全な仕組みと実装

---

**ナビゲーション**

[← 61. Ray Castingと座標系](./61_ray_casting_coordinates_v1.ipynb) | [63. NeRF入門 →](./63_nerf_introduction_v1.ipynb)